Option chains
=======

In [247]:
from ib_insync import *
util.startLoop()
import random
ib = IB()
if(ib.isConnected()):
    ib.disconnect()
random_id = random.randint(0, 9999)
ib.connect('127.0.0.1', 7496, clientId=random_id)

<IB connected to 127.0.0.1:7496 clientId=6197>

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [248]:
#spx = Stock('spx','ARCA','USD')
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

To avoid issues with market data permissions, we'll use delayed data:

In [249]:
ib.reqMarketDataType(4)

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [250]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2022, 11, 11, 10, 33, 40, 154370, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, bid=-1.0, bidSize=0.0, ask=-1.0, askSize=0.0, last=3954.34, lastSize=1.0, close=3956.37)

Take the current market value of the ticker:

In [251]:
spxValue = ticker.marketPrice()
if(str(spxValue)=="nan"):
    spxValue=395
spxValue

3954.34

The following request fetches a list of option chains:

In [252]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"[20221117, 20221215, 20230119, 20230216, 20230...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
1,CBOE,416904,SPXW,100,"[20221111, 20221114, 20221115, 20221116, 20221...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
2,SMART,416904,SPXW,100,"[20221111, 20221114, 20221115, 20221116, 20221...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
3,CBOE,416904,SPX,100,"[20221117, 20221215, 20230119, 20230216, 20230...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."


These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the monthly options trading on SMART:

In [253]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

StopIteration: 

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [ ]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']
print(strikes,expirations)
contracts = [Option('spx', expiration, strike, right, 'SMART', tradingClass='spx')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

[380.0, 385.0, 390.0, 395.0, 400.0, 405.0, 410.0, 415.0] ['20221111', '20221114', '20221116']


48

In [ ]:
contracts[0]

Option(conId=588757101, symbol='SPY', lastTradeDateOrContractMonth='20221111', strike=380.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   221111P00380000', tradingClass='SPY')

Now to get the market data for all options in one go:

In [ ]:
tickers = ib.reqTickers(*contracts)

tickers[0]


Ticker(contract=Option(conId=588757101, symbol='SPY', lastTradeDateOrContractMonth='20221111', strike=380.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   221111P00380000', tradingClass='SPY'), time=datetime.datetime(2022, 11, 11, 9, 16, 16, 2715, tzinfo=datetime.timezone.utc), marketDataType=2, minTick=0.01, bid=0.08, bidSize=13528.0, ask=0.09, askSize=4814.0, prevBid=-1.0, prevBidSize=0.0, prevAsk=-1.0, prevAskSize=0.0, volume=0.0, close=0.09, halted=0.0, modelGreeks=OptionComputation(tickAttrib=0, impliedVol=0.294962624416045, delta=-0.00023716141785525075, optPrice=0.00026096300730892843, pvDividend=0.0, gamma=0.00020534685159968419, vega=0.00017596813503638938, theta=-0.00026096300730892843, undPrice=394.55), bboExchange='c70003', snapshotPermissions=3)

In [ ]:
import numpy as np
def closest_value(input_list, input_value):
       return min(input_list, key=lambda x:abs(x-input_value))

In [ ]:
d=[tick.modelGreeks.delta for tick in tickers]
print(d)
dlta=closest_value(d,0.1)
print("closest",dlta)



[-0.00023716141785525075, -0.004107927363028777, -0.08978048573563203, -0.551687897552121, -0.9687895934918702, -0.999999999999999, -0.999999999999993, -0.999999999999999, -0.04700325737266877, -0.12217736761497541, -0.27495405073722395, -0.513886037138811, -0.7707983605377371, -0.9414711357605545, -0.9988403841817388, -0.9999999999999705, -0.10349234944463265, -0.19579107889886793, -0.33209282843839844, -0.504302408453249, -0.6845717706825616, -0.8390843377158149, -0.9375997319791867, -0.9851132029396067, 0.9997629464512606, 0.9958940562220856, 0.910282783500496, 0.44894552571780283, 0.034171251934880784, 0.00013055466608235272, 4.808832995146656e-07, 1.5072871388190307e-08, 0.9530668270789716, 0.8780512586853436, 0.7258042633232644, 0.4880683133193608, 0.2338114422519226, 0.06795430539497824, 0.014492885091928169, 0.0034560495662559467, 0.8967249018486602, 0.8047134527537193, 0.6690292512924798, 0.4978161187591509, 0.31922378605752544, 0.16744281829804156, 0.07237529842157851, 0.0282

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [ ]:
ib.disconnect()

Error 1100, reqId -1: Connectivity between IB and TWS has been lost.
Error 1100, reqId -1: Connectivity between IB and TWS has been lost.
Error 1100, reqId -1: Connectivity between IB and TWS has been lost.
Error 1100, reqId -1: Connectivity between IB and TWS has been lost.
Error 1100, reqId -1: Connectivity between IB and TWS has been lost.
Error 1100, reqId -1: Connectivity between IB and TWS has been lost.
Error 1100, reqId -1: Connectivity between IB and TWS has been lost.
Error 1100, reqId -1: Connectivity between IB and TWS has been lost.
Error 1100, reqId -1: Connectivity between IB and TWS has been lost.
Error 1102, reqId -1: Connectivity between IB and TWS has been restored - data maintained. All data farms are connected: eufarmnj; cashfarm; usfuture; usfarm.nj; eufarm; usfarm; euhmds; hkhmds; cashhmds; fundfarm; ushmds; secdefnj.
Error 1102, reqId -1: Connectivity between IB and TWS has been restored - data maintained. All data farms are connected: eufarmnj; cashfarm; usfutu